In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [16]:
import matplotlib.pyplot as plt
import glob
import librosa
import numpy as np
import cv2 

normal_xray=glob.glob('/kaggle/input/labeled-chest-xray-images/chest_xray/train/NORMAL/*.jpeg')
pneumonia_xray=glob.glob('/kaggle/input/labeled-chest-xray-images/chest_xray/train/PNEUMONIA/*.jpeg')

data = []
labels = []

image_size=(64,64)

for i in normal_xray:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    data.append(image)
    labels.append(0)

for i in pneumonia_xray:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    data.append(image)
    labels.append(1)
    


    
X=np.array(data)
y=np.array(labels)
y.shape

(5232,)

In [17]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [18]:
X_train.shape

(3348, 64, 64)

In [22]:
normal_xray_test=glob.glob('/kaggle/input/labeled-chest-xray-images/chest_xray/test/NORMAL*.jpeg')
pneumonia_xray_test=glob.glob('/kaggle/input/labeled-chest-xray-images/chest_xray/test/PNEUMONIA*.jpeg')

test_data=[]
test_labels=[]

image_size=(64,64)

for i in normal_xray_test:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    data.append(image)
    labels.append(0)

for i in pneumonia_xray_test:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    test_data.append(image)
    test_labels.append(1)
    


    
X_last_test=np.array(data)
y_last_test=np.array(labels)
y_last_test.shape

(5232,)

In [19]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [20]:
# Build the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(64,64)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Monitor validation loss for learning rate reduction
    factor=0.1,            # Reduce learning rate by a factor of 0.1
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6            # Minimum learning rate
)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model_xray.h5', monitor='val_loss', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
# Train the model
model.fit(X_train, y_train,validation_data=(X_val,y_val),epochs=20, batch_size=32,verbose=1,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Make predictions on new images
predictions = model.predict(X_test[:10])
predicted_classes = np.argmax(predictions, axis=1)

# Print the predicted class and corresponding true class
for i in range(len(predicted_classes)):
    print(f"Predicted class: {predicted_classes[i]}, True class: {np.argmax(y_test[i])}")

Epoch 1/20
105/105 [==============================] - 2s 7ms/step - loss: 24.3294 - accuracy: 0.7082 - val_loss: 0.6794 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 2/20
105/105 [==============================] - 0s 5ms/step - loss: 1.2458 - accuracy: 0.7491 - val_loss: 0.6598 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 3/20
105/105 [==============================] - 1s 5ms/step - loss: 0.6469 - accuracy: 0.7491 - val_loss: 0.6432 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 4/20
105/105 [==============================] - 0s 5ms/step - loss: 0.6308 - accuracy: 0.7491 - val_loss: 0.6296 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 5/20
105/105 [==============================] - 0s 5ms/step - loss: 0.6175 - accuracy: 0.7491 - val_loss: 0.6181 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 6/20
105/105 [==============================] - 0s 5ms/step - loss: 0.6065 - accuracy: 0.7491 - val_loss: 0.6088 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 7/20
105/105 [==============================] - 1s 5ms/step - l

In [23]:
# Evaluate the model
loss, accuracy = model.evaluate(X_last_test, y_last_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Make predictions on new images
predictions = model.predict(X_last_test[:10])
predicted_classes = np.argmax(predictions, axis=1)

# Print the predicted class and corresponding true class
for i in range(len(predicted_classes)):
    print(f"Predicted class: {predicted_classes[i]}, True class: {np.argmax(y_last_test[i])}")

164/164 [==============================] - 0s 2ms/step - loss: 0.5712 - accuracy: 0.7420
Test loss: 0.5712
Test accuracy: 0.7420
1/1 [==============================] - 0s 22ms/step
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
Predicted class: 0, True class: 0
